### Task 1: Install the python package

Starting from 0.3.0, python-woc supports the HTTP API! You are not limited by the access to PKU or UTK servers. 
First, let us install or upgrade the python package:

In [1]:
!python3 -m pip install -U python-woc

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple


### Task 2 (Optional): Generate an API key

This is an optional step but I recommend you to do it. By default HTTP API restricts the number of requests per minute to avoid abuse. To raise the limit, you can generate an API key on the World of Code website.

Currently the website is at: https://woc.osslab-pku.org/ (After we figure out the domain transfer, it will probably be moved to https://worldofcode.org/)

The API key is a string like `woc-XXXXXXXXXXXXXX-YYYYYYYYYYYYYY`, and you pass it to the client with the `api_key` argument:

```python
# sync client
woc = WocMapsRemote(
    base_url="https://woc.osslab-pku.org/api/",
    api_key="woc-XXXXXXXXXXXXXX-YYYYYYYYYYYYYY"
)
# async client
woca = WocMapsRemoteAsync(
    base_url="https://woc.osslab-pku.org/api/",
    api_key="woc-XXXXXXXXXXXXXX-YYYYYYYYYYYYYY"
)
```

### Task 3: Use the python package

The sync remote client feels the same as the local client, and most APIs will just work:

In [2]:
from woc.remote import WocMapsRemote

woc = WocMapsRemote(
    base_url="https://woc.osslab-pku.org/api/",  # <- may be different for you
)
[(m.name, m.version) for m in woc.maps]

[('c2fbb', 'V'),
 ('obb2cf', 'V'),
 ('bb2cf', 'V'),
 ('a2f', 'V'),
 ('a2f', 'T'),
 ('b2A', 'U'),
 ('b2a', 'U'),
 ('A2f', 'V'),
 ('P2a', 'V'),
 ('b2P', 'V'),
 ('b2f', 'V'),
 ('a2P', 'V'),
 ('a2P', 'T'),
 ('b2fa', 'V'),
 ('b2tac', 'V'),
 ('c2p', 'V3'),
 ('c2p', 'V'),
 ('c2pc', 'U'),
 ('c2cc', 'V'),
 ('c2rhp', 'U'),
 ('p2a', 'V'),
 ('ob2b', 'U'),
 ('A2a', 'V'),
 ('A2a', 'U'),
 ('A2a', 'T'),
 ('A2a', 'S'),
 ('a2A', 'V0'),
 ('a2A', 'V3'),
 ('a2A', 'V'),
 ('a2A', 'T'),
 ('a2A', 'S'),
 ('c2dat', 'V'),
 ('c2dat', 'U'),
 ('a2c', 'V'),
 ('a2fb', 'T'),
 ('a2fb', 'S'),
 ('P2c', 'V'),
 ('P2c', 'U'),
 ('c2r', 'T'),
 ('c2r', 'S'),
 ('P2p', 'V'),
 ('P2p', 'U'),
 ('P2p', 'T'),
 ('P2p', 'S'),
 ('P2p', 'R'),
 ('c2h', 'T'),
 ('c2h', 'S'),
 ('c2P', 'V'),
 ('c2P', 'U'),
 ('p2P', 'V'),
 ('p2P', 'U'),
 ('p2P', 'T'),
 ('p2P', 'S'),
 ('p2P', 'R'),
 ('A2c', 'V'),
 ('A2c', 'U'),
 ('A2P', 'V'),
 ('A2P', 'U'),
 ('P2A', 'V'),
 ('P2A', 'U'),
 ('a2p', 'V'),
 ('a2p', 'U'),
 ('a2p', 'T'),
 ('A2b', 'V'),
 ('A2b', 'U'),
 

In [3]:
# get_values API

woc.get_values('c2ta', 
               woc.get_values('c2pc', '009d7b6da9c4419fe96ffd1fffb2ee61fa61532a')[0])

['1092637858', 'Maxim Konovalov <maxim@FreeBSD.org>']

In [4]:
# show_content API

woc.show_content('commit', '009d7b6da9c4419fe96ffd1fffb2ee61fa61532a')

['464ac950171f673d1e45e2134ac9a52eca422132',
 ['dddff9a89ddd7098a1625cafd3c9d1aa87474cc7'],
 ['Warner Losh <imp@FreeBSD.org>', '1092638038', '+0000'],
 ['Warner Losh <imp@FreeBSD.org>', '1092638038', '+0000'],
 "Don't need to declare cbb module.  don't know why I never saw\nduplicate messages..\n"]

The only exception is `all_keys` API, which is not supported by the remote client (I did not find a way to paginate that.)

In [5]:
# Objects API

from woc.objects import *
init_woc_objects(woc)

Commit('009d7b6da9c4419fe96ffd1fffb2ee61fa61532a').parents[0].author

Author(Maxim Konovalov <maxim@FreeBSD.org>)

In [6]:
woc.all_keys('c2p')

NotImplementedError: all_keys is not implemented in WoC HTTP API. If you feel it is necessary, please create a feature request at https://github.com/ssc-oscar/python-woc/issues/new

### Task 4: Batching

Git objects are typically small, and sending dozens of small queries is not efficient. The remote client supports batching by `show_content_many` and `get_values_many`, it will send 10 queries in one request. For the impatient, it displays a progress bar with `progress=True`. The return value is a tuple of 2 dictionaries { results }, { errors }.


In [7]:
woc.show_content_many('commit', woc.get_values('a2c', 'Audris Mockus <audris@utk.edu>')[:50], progress=True)

100%|██████████| 5/5 [00:00<00:00,  5.45it/s]


({'001ec7302de3b07f32669a1f1faed74585c8a8dc': ['d0074dfdf50faf1a679a293d1833af74513d5b38',
   ['13710ca2439f85eff9922169a4588da64b3f1fce'],
   ['Audris Mockus <audris@utk.edu>', '1514659483', '-0500'],
   ['Audris Mockus <audris@utk.edu>', '1514659483', '-0500'],
   'work on diff performance\n'],
  '0037d5c34c2787f2a0b619c5d2a1f76254ac974c': ['ac14b680a6c58f50221b8da7cfa307528b5b971a',
   ['87ec9a9e6fda18cdcb8bd78a0e909afd0e40d329',
    '5fee5205917fa803036a17aba185a0a8af17d1fa'],
   ['Audris Mockus <audris@utk.edu>', '1629765554', '-0400'],
   ['GitHub <noreply@github.com>', '1629765554', '-0400'],
   'Merging 43\n\n'],
  '003f2b790d6fb83924649d90867f3d1545ea0e36': ['3eda5f06cba2c0051367de6ebcf1daf9c3a9cdc6',
   ['8750edd4576f6a0b592a36d777f09d272c42097b',
    'a4ac9e07db0a27268584d2912ddf2cceaf3dc3d2'],
   ['Audris Mockus <audris@utk.edu>', '1512787832', '-0500'],
   ['GitHub <noreply@github.com>', '1512787832', '-0500'],
   'Merge pull request #1 from dylanrainwater/master\n\nCreate

In [9]:
woc.get_values_many('c2b', woc.get_values('P2c', 'user2589_minicms')[:50], progress=True)


100%|██████████| 5/5 [00:00<00:00,  5.56it/s]


({'05cf84081b63cda822ee407e688269b494a642de': ['03d1977aecf31666578422805c60cf61562ceea1',
   '1619cce13ffcc3eaaddb1f714072914625f576f6',
   '1838ded6411e5fbfd9d0168de007de3e78e94d94',
   '3993b20337e33a36c9125d139f1f53a279a4c128',
   '3dd682cbf7fd0c482d31f0e74e9ed05e4853cd9f',
   '44a07afd30f499cdba30847094a1e92f13e1320e',
   '6ad59f8158da5afca559c5b3d422af2b1a17eb81',
   '6b0af6a378d95ac9a11297fe83baca147c7af4c2',
   '70ce71f3cd86c10b11b778e502ca9364b2262d8f',
   '83d5c112f8584c5a7f2db377e5dda2216586f2ca',
   '9599aebf9b3cae84678ef0703e6217d47030b0ff',
   'dae40a15a0f5eaef5259d66defe3544166da59bd',
   'e8b638f1548c5b74e2bb4b74d3aaf8da93e24aa1'],
  '086a622f0e24feb7853c520f965f04c7fc7e4861': ['773e50f6785dafd0acbe050e7dd16a8179297652',
   'ad4d743e78a5ceb39942675cf968c1dcaa935557',
   'e87238845f6b48d16807cb4d56a58bd17ab41931'],
  '0ad5b0b392ed22cef866de5ae8504462183b0316': ['29f38813ea514935bce39d8b24c31e486d033340'],
  '104b8284ba6435a3c07eb5ce82f15cb0f956eda3': ['c11edc429d433037a1

### Task 5: Go Async

The remote client also supports async API, which is useful when you are running multiple requests in parallel. APIs are similar to the sync ones, but with `await` in front of them.

In [ ]:
from woc.remote import WocMapsRemoteAsync, WocMapsRemote

woca = WocMapsRemoteAsync(
    base_url="https://woc.osslab-pku.org/api/",
)
[(m.name, m.version) for m in await woca.get_maps()]

[('c2fbb', 'V'),
 ('obb2cf', 'V'),
 ('bb2cf', 'V'),
 ('a2f', 'V'),
 ('a2f', 'T'),
 ('b2A', 'U'),
 ('b2a', 'U'),
 ('A2f', 'V'),
 ('P2a', 'V'),
 ('b2P', 'V'),
 ('b2f', 'V'),
 ('a2P', 'V'),
 ('a2P', 'T'),
 ('b2fa', 'V'),
 ('b2tac', 'V'),
 ('c2p', 'V3'),
 ('c2p', 'V'),
 ('c2pc', 'U'),
 ('c2cc', 'V'),
 ('c2rhp', 'U'),
 ('p2a', 'V'),
 ('ob2b', 'U'),
 ('A2a', 'V'),
 ('A2a', 'U'),
 ('A2a', 'T'),
 ('A2a', 'S'),
 ('a2A', 'V0'),
 ('a2A', 'V3'),
 ('a2A', 'V'),
 ('a2A', 'T'),
 ('a2A', 'S'),
 ('c2dat', 'V'),
 ('c2dat', 'U'),
 ('a2c', 'V'),
 ('a2fb', 'T'),
 ('a2fb', 'S'),
 ('P2c', 'V'),
 ('P2c', 'U'),
 ('c2r', 'T'),
 ('c2r', 'S'),
 ('P2p', 'V'),
 ('P2p', 'U'),
 ('P2p', 'T'),
 ('P2p', 'S'),
 ('P2p', 'R'),
 ('c2h', 'T'),
 ('c2h', 'S'),
 ('c2P', 'V'),
 ('c2P', 'U'),
 ('p2P', 'V'),
 ('p2P', 'U'),
 ('p2P', 'T'),
 ('p2P', 'S'),
 ('p2P', 'R'),
 ('A2c', 'V'),
 ('A2c', 'U'),
 ('A2P', 'V'),
 ('A2P', 'U'),
 ('P2A', 'V'),
 ('P2A', 'U'),
 ('a2p', 'V'),
 ('a2p', 'U'),
 ('a2p', 'T'),
 ('A2b', 'V'),
 ('A2b', 'U'),
 

In [24]:
# 1. get_values API

# woc.get_values('c2ta', 
#                woc.get_values('c2pc', '009d7b6da9c4419fe96ffd1fffb2ee61fa61532a')[0])

await woca.get_values('c2ta', 
               (await woca.get_values('c2pc', '009d7b6da9c4419fe96ffd1fffb2ee61fa61532a'))[0])

['1092637858', 'Maxim Konovalov <maxim@FreeBSD.org>']

In [ ]:
async for i in woca.iter_values("P2c", "user2589_minicms"):
    print(i)

['05cf84081b63cda822ee407e688269b494a642de', '086a622f0e24feb7853c520f965f04c7fc7e4861', '0ad5b0b392ed22cef866de5ae8504462183b0316', '104b8284ba6435a3c07eb5ce82f15cb0f956eda3', '1837bfa6553a9f272c5dcc1f6259ba17357cf8ed', '19ddf6dafb6014c954253bd022778051213ccd9a', '1d3038eab8cac1e8a9df187d411fbc0e4a317270', '1e971a073f40d74a1e72e07c682e1cba0bae159b', '1eda863abed481df83c680a6c31fad05719b166b', '27f6af62ff6facfb21a7fe33cddfc115f93cb75f', '2881cf0080f947beadbb7c240707de1b40af2747', '2c02c1f9b1a959c5228bf8cfad1a09fd5489b381', '3303b05caf2f9b51fc6323820fe9e04780c40e48', '335aeff4c90d4d31562a24b2648ed529ef664664', '3b0cbf364870cd35d9e41630387d97393fea2fa5', '3beec34c51d9ae5d60c7eb976bb03a95db235514', '3c57cd4791ac46ccb73cc22a50d9a4c77e5cd0a3', '3c59a5aca8ee3e201977558fe9f1ea5489d2b1b3', '3f68ba216c938e93aff1dc45b241511a0fa94e51', '43981b68b7a24544d4bc4f3094be7a12c9f0afe0', '4dffda766eba4f4edc31eb0b7691cc75d7775de0', '58898751944b69ffc04d148b0917473e2d5d5db8', '5b0afd26ed90f8b3352f4ac8a53da9

In [ ]:
await woca.show_content("tree", "f1b66dcca490b5c4455af319bc961a34f69c72c2")

[['100644', 'README.md', '05fe634ca4c8386349ac519f899145c75fff4169'],
 ['100644', 'course.pdf', 'dfcd0359bfb5140b096f69d5fad3c7066f101389']]